In [1]:
from binance_trader import BinanceTrader
import os
from dotenv import load_dotenv
import pandas as pd

load_dotenv()

API_KEY = os.getenv("API_KEY")
SECRET_KEY = os.getenv("SECRET_KEY")

trader = BinanceTrader(API_KEY, SECRET_KEY)
idx = pd.IndexSlice

In [19]:
params = {"symbol" : ["BTCUSDT", "ETHUSDT"], "interval" : "1m", "start_dt" : "2024-11-01", "end_dt" : "2024-12-22"}


df = trader.get_candlestick_data_range(**params)
df.head(10)

Open      High       Low     Close   Volume  \
Open Time           Symbol                                                     
2024-11-05 13:24:00 BTCUSDT  68710.01  68789.68  68710.01  68736.92  0.29141   
                    ETHUSDT   2438.40   2440.11   2438.40   2440.11  3.47320   
2024-11-05 13:25:00 BTCUSDT  68736.00  68755.48  21250.00  68750.26  0.32806   
                    ETHUSDT   2438.81   2439.92   2438.81   2439.53  3.22890   
2024-11-05 13:26:00 BTCUSDT  68746.87  69039.27  68666.48  68748.01  0.48238   
                    ETHUSDT   2439.55   2441.00   2439.23   2439.23  3.01840   
2024-11-05 13:27:00 BTCUSDT  68747.00  68757.98  65187.77  68750.15  0.27751   
                    ETHUSDT   2439.01   2440.49   2439.01   2440.02  3.43380   
2024-11-05 13:28:00 BTCUSDT  68750.36  68781.63  16275.00  68778.00  0.62608   
                    ETHUSDT   2440.20   2441.16   2439.82   2440.78  5.01540   

                             Quote Asset Volume  Number of Trades  \
Open Time           Symbol                                          
2024-11-05 13:24:00 BTCUSDT        20032.192563              56.0   
                    ETHUSDT         8473.003310              27.0   
2024-11-05 13:25:00 BTCUSDT        22434.253383              73.0   
                    ETHUSDT         7876.714896              36.0   
2024-11-05 13:26:00 BTCUSDT        33174.664570             131.0   
                    ETHUSDT         7365.502186              35.0   
2024-11-05 13:27:00 BTCUSDT        19075.839229              63.0   
                    ETHUSDT         8377.433080              37.0   
2024-11-05 13:28:00 BTCUSDT        35322.487892             125.0   
                    ETHUSDT        12240.136679              45.0   

                             Taker Buy Base Asset Volume  \
Open Time           Symbol                                 
2024-11-05 13:24:00 BTCUSDT                      0.29141   
                    ETHUSDT                      3.17170   
2024-11-05 13:25:00 BTCUSDT                      0.10787   
                    ETHUSDT                      3.04850   
2024-11-05 13:26:00 BTCUSDT                      0.30178   
                    ETHUSDT                      2.31740   
2024-11-05 13:27:00 BTCUSDT                      0.05967   
                    ETHUSDT                      3.24930   
2024-11-05 13:28:00 BTCUSDT                      0.21250   
                    ETHUSDT                      2.46060   

                             Taker Buy Quote Asset Volume  
Open Time           Symbol                                 
2024-11-05 13:24:00 BTCUSDT                  20032.192563  
                    ETHUSDT                   7737.543536  
2024-11-05 13:25:00 BTCUSDT                   7415.894900  
                    ETHUSDT                   7436.569564  
2024-11-05 13:26:00 BTCUSDT                  20755.789572  
                    ETHUSDT                   5655.123723  
2024-11-05 13:27:00 BTCUSDT                   4102.321752  
                    ETHUSDT                   7927.232785  
2024-11-05 13:28:00 BTCUSDT                  14603.069644  
                    ETHUSDT                   6005.879181

In [23]:
for lag in [1,2,3,6,12]:
    for symbol in params["symbol"]:
        df.loc[idx[:,symbol], f"Target_{lag}"] = df.loc[idx[:,symbol], "Close"].pct_change(lag).shift(-lag)
        df.loc[idx[:,symbol], f"Return_{lag}"] = df.loc[idx[:,symbol], "Close"].pct_change(lag)
        
df.dropna(inplace=True)
df.head(26)

Open      High       Low     Close   Volume  \
Open Time           Symbol                                                     
2024-11-05 13:48:00 BTCUSDT  68776.01  68807.00  68775.50  68807.00  0.34538   
                    ETHUSDT   2443.09   2444.61   2442.61   2444.61  5.06640   
2024-11-05 13:49:00 BTCUSDT  68807.00  68833.67  68799.31  68833.67  0.29877   
                    ETHUSDT   2444.64   2445.87   2444.64   2445.64  4.26430   
2024-11-05 13:50:00 BTCUSDT  68835.70  68835.70  68814.01  68824.25  0.08711   
                    ETHUSDT   2445.23   2445.24   2444.65   2444.80  2.18410   
2024-11-05 13:51:00 BTCUSDT  68825.09  68825.41  68770.01  68785.20  0.24180   
                    ETHUSDT   2444.30   2444.30   2442.01   2442.29  3.85920   
2024-11-05 13:52:00 BTCUSDT  68782.40  68850.00  68766.30  68810.28  0.29780   
                    ETHUSDT   2442.14   2444.01   2441.20   2441.61  2.65840   
2024-11-05 13:53:00 BTCUSDT  68806.00  68868.00  68739.78  68868.00  0.28636   
                    ETHUSDT   2441.46   2443.59   2441.01   2443.59  5.04040   
2024-11-05 13:54:00 BTCUSDT  68868.00  68902.00  68810.32  68810.32  0.17503   
                    ETHUSDT   2443.80   2444.77   2442.01   2442.11  5.51710   
2024-11-05 13:55:00 BTCUSDT  68810.32  68822.56  68810.31  68818.27  0.46752   
                    ETHUSDT   2442.20   2443.09   2442.20   2442.53  4.00670   
2024-11-05 13:56:00 BTCUSDT  68820.00  68822.56  68797.51  68799.99  0.17156   
                    ETHUSDT   2442.35   2442.57   2442.06   2442.20  3.04090   
2024-11-05 13:57:00 BTCUSDT  68800.00  68827.99  65000.00  68827.99  0.76398   
                    ETHUSDT   2442.24   2498.92   2442.11   2442.83  6.94800   
2024-11-05 13:58:00 BTCUSDT  68827.99  68870.01  68827.99  68863.99  0.40164   
                    ETHUSDT   2442.86   2443.70   2442.82   2442.86  2.08800   
2024-11-05 13:59:00 BTCUSDT  68846.00  68846.00  68821.67  68821.67  0.21430   
                    ETHUSDT   2442.33   2443.22   2442.33   2442.38  3.60390   
2024-11-05 14:00:00 BTCUSDT  68819.51  68820.01  65000.00  68801.01  0.38092   
                    ETHUSDT   2441.64   2442.28   2441.57   2441.72  2.49720   

                             Quote Asset Volume  Number of Trades  \
Open Time           Symbol                                          
2024-11-05 13:48:00 BTCUSDT        23760.374447              85.0   
                    ETHUSDT        12380.973692              60.0   
2024-11-05 13:49:00 BTCUSDT        20559.527369              71.0   
                    ETHUSDT        10426.960257              46.0   
2024-11-05 13:50:00 BTCUSDT         5995.170702              26.0   
                    ETHUSDT         5339.809467              28.0   
2024-11-05 13:51:00 BTCUSDT        16633.391164              85.0   
                    ETHUSDT         9427.293712              46.0   
2024-11-05 13:52:00 BTCUSDT        20494.075771              75.0   
                    ETHUSDT         6491.587063              29.0   
2024-11-05 13:53:00 BTCUSDT        19710.489684              89.0   
                    ETHUSDT        12308.191157              57.0   
2024-11-05 13:54:00 BTCUSDT        12055.411219              52.0   
                    ETHUSDT        13483.501810              64.0   
2024-11-05 13:55:00 BTCUSDT        32175.280955             113.0   
                    ETHUSDT         9787.207429              45.0   
2024-11-05 13:56:00 BTCUSDT        11805.705238              39.0   
                    ETHUSDT         7426.730984              36.0   
2024-11-05 13:57:00 BTCUSDT        52398.246710             137.0   
                    ETHUSDT        16973.774087              75.0   
2024-11-05 13:58:00 BTCUSDT        27646.209386              86.0   
                    ETHUSDT         5101.788338              23.0   
2024-11-05 13:59:00 BTCUSDT        14751.073862              65.0   
                    ETHUSDT         8803.860198              39.0  